## Convergence and Stability

Let's put these ideas to use to design a numerical scheme whose solution should approximate the solution of a PDE. Along the way, we'll need to do some work to guarantee that the numerical solution actually converges to the real solution. 